In [ ]:
import tensorflow as tf
import serial
import numpy as np
from pythonosc import osc_message_builder
from pythonosc import udp_client
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [ ]:
RANDOM_SEED = 42
tf.set_random_seed(RANDOM_SEED)

img_size_flat=4;
num_classes = 3;
x = tf.placeholder(tf.float32, [None, img_size_flat])
y_true = tf.placeholder(tf.float32, [None, num_classes])
y_true_cls = tf.placeholder(tf.int64, [None])
weights = tf.Variable(tf.zeros([img_size_flat, num_classes]))
biases = tf.Variable(tf.zeros([num_classes]))

logits = tf.matmul(x, weights) + biases
y_pred = tf.nn.softmax(logits)
y_pred_cls = tf.argmax(y_pred, axis=1)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                        labels=y_true)

cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(cost)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

session = tf.Session()
session.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.restore(session, './Neural_Chords/trained')

In [ ]:
def PlayChord(Note):
    if Note == 0:
        base = 60 
    elif  Note == 1:
        base = 65
    elif Note == 2:
        base = 67
    fourth = base+4
    seventh = base+7
    vel = 127
    chord_as_message = str(base)+' '+str(vel)+' '+str(fourth)+' '+str(vel)+' '+str(seventh)+' '+str(vel)
    return chord_as_message

In [ ]:
arduinoSerialData = serial.Serial('com6',9600)
client = udp_client.UDPClient('127.0.0.1', 8000)
msg = osc_message_builder.OscMessageBuilder(address = '/glove')
while 1==1:
     #Create Serial port object called arduinoSerialData
    Hand=np.zeros([1,4])
    j = 0;
    while(j<4):
        Hand[0,j] = int(arduinoSerialData.readline());
        j=j+1;
    output = session.run(y_pred_cls, {x:Hand})
    print(PlayChord(output[0]))

arduinoSerialData.close()

In [ ]:
arduinoSerialData.close()